In [1]:
# capture --no-stderr
%pip install -U langgraph langsmith

# Used for this tutorial; not a requirement for LangGraph
%pip install -U langchain-openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("API_BASE_URL")

OPENAI_API_KEY: ········
API_BASE_URL: ········


In [3]:
_set_env("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "LangGraph Tutorial"

LANGSMITH_API_KEY: ········


In [4]:
_set_env("TAVILY_API_KEY")

TAVILY_API_KEY: ········


In [27]:
from typing import Annotated

from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict
from langgraph.checkpoint.sqlite import SqliteSaver

from langgraph.graph import StateGraph,START
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

memory = SqliteSaver.from_conn_string(":memory:")


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


tool = TavilySearchResults(max_results=2)
tools = [tool]
api_key = os.environ.get("OPENAI_API_KEY")
base_url = os.environ.get("API_BASE_URL")

llm = ChatOpenAI(model="gpt-4o-mini",openai_api_key=api_key, openai_api_base=base_url)
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

In [28]:
graph = graph_builder.compile(
    checkpointer=memory,
    # This is new!
    interrupt_before=["tools"],
    # Note: can also interrupt __after__ actions, if desired.
    # interrupt_after=["tools"]
)

In [29]:
user_input = "我正在学习 LangGraph，你能提供一些学习资料给我吗？"
config = {"configurable": {"thread_id": "3"}}
# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

我正在学习 LangGraph，你能提供一些学习资料给我吗？
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_DnhGKSHuFfVYbzwvsp0YVTIf)
 Call ID: call_DnhGKSHuFfVYbzwvsp0YVTIf
  Args:
    query: LangGraph 学习资料


In [30]:
snapshot = graph.get_state(config)
snapshot.next

('tools',)

In [31]:
existing_message = snapshot.values["messages"][-1]
existing_message.tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'LangGraph 学习资料'},
  'id': 'call_DnhGKSHuFfVYbzwvsp0YVTIf',
  'type': 'tool_call'}]

In [32]:
# `None` will append nothing new to the current state, letting it resume as if it had never been interrupted
events = graph.stream(None, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================= Tool Message =================================
Name: tavily_search_results_json

[{'url': 'https://github.com/langchain-ai/langgraph', 'content': 'LangGraph is a library for building stateful, multi-actor applications with LLMs, used to create agent and multi-agent workflows. Compared to other LLM frameworks, it offers these core benefits: cycles, controllability, and persistence. LangGraph allows you to define flows that involve cycles, essential for most agentic architectures, differentiating it from DAG-based solutions.'}, {'url': 'https://www.langchain.com/langgraph', 'content': "LangGraph sets the foundation for how we can build and scale AI workloads — from conversational agents, complex task automation, to custom LLM-backed experiences that 'just work'. The next chapter in building complex production-ready features with LLMs is agentic, and with LangGraph and LangSmith, LangChain delivers an out-of-the-box solution ..."}]


BadRequestError: Error code: 400 - {'error': {'message': "Missing required parameter: 'messages[2].content[0].type'. (request id: 202407230551306141665345677042)", 'type': 'invalid_request_error', 'param': 'messages[2].content[0].type', 'code': 'missing_required_parameter'}}